# Module

In [8]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X), net(X).shape

(tensor([[-0.0856, -0.1357,  0.1590, -0.0049, -0.1928, -0.0939, -0.0392,  0.1506,
          -0.0701,  0.2404],
         [-0.0720, -0.0962, -0.0313, -0.0150, -0.2179, -0.0643,  0.0457,  0.1516,
          -0.0656,  0.2112]], grad_fn=<AddmmBackward>),
 torch.Size([2, 10]))

## Any layer/net should be a subclass of Module

In [42]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__() # it keeps all the things in Module
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X))) # F.relu() here--it is a function. nn.ReLU() is a module
    
net = MLP()
net(X)

tensor([[-0.0517, -0.2053, -0.0593,  0.1088, -0.2596,  0.2546, -0.1927, -0.0061,
         -0.2509, -0.1236],
        [ 0.0471, -0.2188,  0.0428, -0.0531, -0.1412,  0.3329, -0.0547, -0.1892,
         -0.1327, -0.0535]], grad_fn=<AddmmBackward>)

In [19]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # adding each layer to self._module
            # self.modules is not assignable, use self._modules instead
            self._modules[block] = block
    def forward(self, X):
        # do not forget '.values()'
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1875, -0.1561, -0.0659,  0.1539,  0.0518, -0.0928,  0.1218, -0.0006,
         -0.1771, -0.0855],
        [-0.2251, -0.1038, -0.0866,  0.0690, -0.0109, -0.1189,  0.0580, -0.0686,
         -0.2471,  0.0243]], grad_fn=<AddmmBackward>)

## Customize your own module--__init__ and forward
You do not need to customize backward function

In [41]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X@self.rand_weight + 1) # 1 is bias
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(0.2035, grad_fn=<SumBackward0>)

In [43]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.3197, grad_fn=<SumBackward0>)

## Customize your own layer

In [118]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X-X.mean()
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [122]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean() # =0

tensor(6.5193e-09, grad_fn=<MeanBackward0>)

### customize parameters in your own layer

In [124]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # all parameters are nn.Parameter
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = X@self.weight.data + self.bias.data
        return F.relu(linear)
dense = MyLinear(5,3)
dense.weight, dense.bias

(Parameter containing:
 tensor([[-0.3247,  1.4145, -0.6740],
         [ 0.5044, -0.6385, -1.0057],
         [ 0.6626, -0.8953,  1.1407],
         [ 0.0283, -0.9799,  0.5313],
         [-0.2233,  0.9526,  0.3469]], requires_grad=True),
 Parameter containing:
 tensor([-0.5914,  0.7070, -0.0969], requires_grad=True))

In [125]:
dense(torch.rand(2,5))

tensor([[0.0000, 0.5470, 0.1752],
        [0.0000, 1.5496, 0.0000]])

In [126]:
net = MySequential(MyLinear(64,8), MyLinear(8,1))
net(torch.rand(2, 64))

tensor([[12.1911],
        [ 6.8850]])

# Parameters in net

In [45]:
X = torch.rand(size=(2,4))

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
net(X)

tensor([[-0.4092],
        [-0.3267]], grad_fn=<AddmmBackward>)

## Print parameters

### print paramters in a layer

In [64]:
# net[0].state_dict()
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.0213, -0.0227,  0.3455, -0.1031, -0.2657,  0.1896,  0.1500,  0.3076]])),
             ('bias', tensor([-0.2987]))])

### print single parameters

In [72]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data) # it is a tensor
print(net[2].bias.grad) # it is the gradience

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2987], requires_grad=True)
tensor([-0.2987])
None


### print all parameter

In [88]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# there is no 1. -- because it is ReLU

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [89]:
# you can access to a single parameter via it name
net.state_dict()['2.bias']

tensor([-0.2987])

### nested-block 

In [98]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1()) # customize the name of module
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[-0.2640],
        [-0.2642]], grad_fn=<AddmmBackward>)

In [100]:
rgnet

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

## Init parameters

In [107]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
# apply--apply function to all modules
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0031, -0.0029,  0.0080, -0.0158]), tensor(0.))

In [108]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [112]:
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data[0])

tensor([ 0.1998, -0.6266,  0.3427,  0.3944])
tensor([42., 42., 42., 42., 42., 42., 42., 42.])


In [113]:
print(net[0].weight.data[0])
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([ 0.1998, -0.6266,  0.3427,  0.3944])


tensor([42.0000,  0.3734,  1.3427,  1.3944])

### Share parameters between layers

In [115]:
# shared layers must be init outside
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 1000
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# Read/Write

## Save tensor

In [127]:
x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [129]:
# save as a list
y = torch.zeros(4)
torch.save([x, y], 'xy-file')

x2, y2 = torch.load('xy-file')
x2, y2

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [132]:
# save as a dict
mydict = {'x':x, 'y':y}
torch.save(mydict, 'xy-file')

mydict2 = torch.load('xy-file')
mydict2, mydict2['x']

({'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])},
 tensor([0, 1, 2, 3]))

## Save model

In [136]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(2,20)
Y = net(X)
Y

tensor([[-0.1014,  0.0279,  0.2560,  0.0087, -0.1491, -0.3087,  0.0819,  0.0854,
         -0.0837,  0.1556],
        [-0.1009,  0.0230,  0.2841, -0.1624, -0.0508, -0.2481,  0.1552,  0.0653,
          0.2642,  0.2398]], grad_fn=<AddmmBackward>)

In [137]:
torch.save(net.state_dict(), 'mlp.params')

In [141]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)